<a href="https://colab.research.google.com/github/Nway-Yadanar/Gardening/blob/main/Gardening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install gradio
!pip install gradio-flatpickr-calendar
!pip install ephem

In [19]:
import json
import requests
def load_json():
    url = "https://raw.githubusercontent.com/Nway-Yadanar/indoor_plant_dataset/refs/heads/main/indoor_plants.json"
    return requests.get(url).json()

plant = load_json()
print("Total number of plants:", len(plant))

Total number of plants: 82


In [20]:
from datetime import datetime
journal_entries = []

def add_entry(date, entry_text):
    if date is None or entry_text.strip() == "":
        return "Please provide a date and an entry.", journal_entries
    journal_entries.append({'date': date, 'entry': entry_text.strip()})
    return f"Added entry for {date}", journal_entries

def format_entries(entries):
    if not entries:
        return "No journal entries yet."
    return "\n\n".join([f"{i+1}. [{e['date']}]: {e['entry']}" for i, e in enumerate(entries)])

def on_add(date, entry_text):
        msg, entries = add_entry(str(date), entry_text)
        return msg, format_entries(entries)

In [21]:

import requests
import ephem # for moon phase
from datetime import datetime





#Moon phase
# def moon_phase(date_str):
#   if isinstance(date_str, str):
#     date_obj = datetime.strptime(date_str, "%Y/%m/%d")
#   else:
#     date_obj = date_str

#   ephem_date = ephem.Date(date_obj)

#     # Lunar age = date - previous new moon
#   prev_new_moon = ephem.previous_new_moon(ephem_date)
#   lunar_age = ephem_date - prev_new_moon  # in days

#   if lunar_age < 1.0:
#         return "New Moon 🌑"
#   elif lunar_age < 6.4:
#         return "Waxing Crescent Moon 🌒"
#   elif lunar_age < 8.4:
#         return "First Quarter 🌓"
#   elif lunar_age < 13.8:
#         return "Waxing Gibbous Moon 🌔"
#   elif lunar_age < 15.8:
#         return "Full Moon 🌕"
#   elif lunar_age < 21.1:
#         return "Waning Gibbous Moon 🌖"
#   elif lunar_age < 23.1:
#         return "Last Quarter 🌗"
#   elif lunar_age <= 29.53:
#         return "Waning Crescent Moon 🌘"
#   else:
#       return "New Moon 🌑"


def moon_phase(date_str):
    date = ephem.Date(date_str)
    moon = ephem.Moon(date)
    phase = moon.phase  # returns percentage illumination (0 to 100)

    if phase < 5:
        return "New Moon"
    elif 5 <= phase < 45:
        return "Waxing Crescent Moon"
    elif 45 <= phase < 55:
        return "First Quarter Moon"
    elif 55 <= phase < 95:
        return "Waxing Gibbous Moon"
    elif 95 <= phase <= 100:
        return "Full Moon"
    elif 85 <= phase < 95:
        return "Waning Gibbous Moon"
    elif 45 <= phase < 55:
        return "Last Quarter Moon"
    elif 5 <= phase < 45:
        return "Waning Crescent Moon"
    else:
        return "Unknown"



#Need to edit this with JSON. manual name remove yan
# def suggest_plants(phase):
#     suggestions = []
#     for p in plant:
#         name = p["plant"]
#         if phase.startswith("New Moon") and any(x in name.lower() for x in ["lettuce", "spinach", "leaf", "basil"]):
#             suggestions.append(name)
#         elif phase.startswith("Full Moon") and any(x in name.lower() for x in ["tomato", "pepper", "capsicum"]):
#             suggestions.append(name)
#         elif phase.startswith("Waxing Crescent Moon") :
#             # suggestions.append("Not a good time to start growing. Instead take care of existing plants")

#         elif phase.startswith("Waning Crescent Moon") and any(x in name.lower() for x in ["leaf"]):
#             suggestions.append(name)
#     return suggestions[:5]


def normalize_phase(phase_str):
    return phase_str.strip().lower()

def suggest_plants(phase, edible_only=True):
    suggestions = []
    phase_norm = normalize_phase(phase)

    for p in plant:
        name = p["plant"]
        moon_info = p.get("moon_phase", {})
        is_edible = p.get("edible", False)
        grow_phase = moon_info.get("growing", "")
        harvest_phase = moon_info.get("harvesting", "")

        grow_phase_norm = normalize_phase(grow_phase)

        if edible_only and not is_edible:
            continue  # skip inedible plants if only edible wanted
        if not edible_only and is_edible:
            continue  # skip edible plants if only inedible wanted


        if phase_norm == grow_phase_norm:
            if is_edible and edible_only and harvest_phase:
                plant_str = f"{name} (Harvest during {harvest_phase})"
            else:
                plant_str = name
            suggestions.append(plant_str)

    return suggestions




# def moon_gardening(date_str):
#     try:
#         # Debug print
#         print("Received date_str:", date_str)

#
#         if isinstance(date_str, datetime):
#             date = date_str
#         else:
#             # Assume it's a string and possibly ISO format with 'T'

#             date = datetime.fromisoformat(date_str.split("T")[0])  # "2025-07-24"

#         print("Parsed date:", date)

#         phase = moon_phase(date.strftime("%Y/%m/%d"))
#         print("Moon phase:", phase)

#         plants = suggest_plants(phase)
#         print("Suggested plants:", plants)

#         moon_phase_str = f"🌙 Moon Phase on {date.strftime('%B %d, %Y')}: {phase}"
#         suggested_plants_str = f"🪴 Suggested Plants to Grow:\n• " + "\n• ".join(plants)

#         return moon_phase_str, suggested_plants_str

#     except Exception as e:
#         error_msg = f"Error: {str(e)}"
#         print(error_msg)
#         return error_msg, error_msg

def moon_gardening(date_str, edible_choice):
    try:
        if isinstance(date_str, datetime):
            date = date_str
        else:
            date_part = date_str.split("T")[0]
            date = datetime.fromisoformat(date_part)

        phase = moon_phase(date.strftime("%Y/%m/%d"))

        # Convert Gradio choice string to boolean
        edible_only = (edible_choice == "Edible")
        plants = suggest_plants(phase, edible_only=edible_only)

        moon_phase_str = f"🌙 Moon Phase on {date.strftime('%B %d, %Y')}: {phase}"

        if not plants:
            if edible_only:
                suggested_plants_str = "Nothing edible should be planted today. Focus on pruning, repotting and care."
            else:
                suggested_plants_str = "Nothing inedible should be planted today. Focus on pruning, repotting and care."
        else:
            suggested_plants_str = "🪴 Suggested Plants to Grow:\n• " + "\n• ".join(plants)



        return moon_phase_str, suggested_plants_str

    except Exception as e:
        return f"Error: {str(e)}", ""





Gradio UI

In [22]:

import gradio as gr
from gradio_flatpickr_calendar import Calendar
from datetime import datetime

def chat_logic(history, user_input):
    history = history + [{"role": "user", "content": user_input}]
    bot_response = f"You said: {user_input}"  # Replace this with real chatbot logic
    history.append({"role": "assistant", "content": bot_response})
    return history, ""



with gr.Blocks() as demo:
    with gr.Row():
        # 🌿 Sidebar on the Left
        with gr.Column(scale=1):
            gr.Markdown("### 🪴 Navigation")
            btn_chatbot = gr.Button("💬 Chatbot")
            btn_growth = gr.Button("🌱 Growth Tracker")
            btn_faq = gr.Button("🌻 Plant FAQ")
            btn_journal = gr.Button("📖 Seed Journal")
            btn_lunar = gr.Button("🌕 Lunar Guide")

        # 🌻 Pages on the Right
        with gr.Column(scale=4) as page_chatbot:
            gr.Markdown("### 💬 Chatbot")
            gr.Markdown("A safe place for your plant discussion and suggestions ")
            chatbot = gr.Chatbot(type="messages", label="Chat Window")
            msg = gr.Textbox(label="Welcome. How can I help you with your gardening?")
            state = gr.State([])
            msg.submit(chat_logic, inputs=[state, msg], outputs=[chatbot, msg])

        with gr.Column(scale=4, visible=False) as page_growth:
            gr.Markdown("### 🌱 Growth Tracker")
            gr.Markdown("Track your baby pot’s growth here.")

        with gr.Column(scale=4, visible=False) as page_faq:
            gr.Markdown("### 🌻 Plant FAQ")
            gr.Markdown("Answers about pests, sunlight needs, and more.")

        with gr.Column(scale=4, visible=False) as page_journal:
            gr.Markdown("### 📖 Seed Journal")
            gr.Markdown("Write and track your planting journey.")

            date_input = Calendar(label="Select Date", value=datetime.today().date(), mode="single")
            text_input = gr.Textbox(lines=4, placeholder="Write your journal entry here...", label="Entry")
            add_button = gr.Button("Add Entry")
            output_text = gr.Textbox(label="Entries", interactive=False)
            status = gr.Textbox(label="Status", interactive=False)
            add_button.click(on_add, inputs=[date_input, text_input], outputs=[status, output_text])

        with gr.Column(scale=4, visible=False) as page_lunar:
            gr.Markdown("### 🌕 Lunar Guide")
            gr.Markdown("Lunar cycle-based gardening tips.")

            lunar_cal = Calendar(label="Select a Date", type="datetime", mode="date")
            edible_choice = gr.Radio(choices=["Edible", "Inedible"], label="Choose Plant Type", value="Edible")
            phase_output = gr.Textbox(label="Moon Phase")
            suggestion_output = gr.Textbox(label="Suggested Plants")

            lunar_cal.change(fn=moon_gardening, inputs=lunar_cal, outputs=[phase_output, suggestion_output])
            edible_choice.change(fn=moon_gardening, inputs=[lunar_cal, edible_choice], outputs=[phase_output, suggestion_output])


#         # 🔁 Navigation Logic
        def switch_to(chat=False, growth=False, faq=False, journal=False, lunar=False):
            return (
                gr.update(visible=chat),
                gr.update(visible=growth),
                gr.update(visible=faq),
                gr.update(visible=journal),
                gr.update(visible=lunar),
            )


        btn_chatbot.click(fn=lambda: switch_to(chat=True),
                          outputs=[page_chatbot, page_growth, page_faq, page_journal, page_lunar])
        btn_growth.click(fn=lambda: switch_to(growth=True),
                          outputs=[page_chatbot, page_growth, page_faq, page_journal, page_lunar])
        btn_faq.click(fn=lambda: switch_to(faq=True),
                          outputs=[page_chatbot, page_growth, page_faq, page_journal, page_lunar])
        btn_journal.click(fn=lambda: switch_to(journal=True),
                          outputs=[page_chatbot, page_growth, page_faq, page_journal, page_lunar])
        btn_lunar.click(fn=lambda: switch_to(lunar=True),
                          outputs=[page_chatbot, page_growth, page_faq, page_journal, page_lunar])




/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 2 arguments for function <function moon_gardening at 0x7d176eb2ede0>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 2 arguments for function <function moon_gardening at 0x7d176eb2ede0>, received 1.
  warnings.warn(


1. Plant FAQ

In [23]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e76d395a5a2ec318d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:


print(moon_phase("2025-07-05"))  # likely Full Moon 🌕
print(moon_phase("2025-07-12"))  # Waning Gibbous 🌖
print(moon_phase("2025-07-19"))  # Near New Moon 🌑
print(moon_phase("2025-07-21"))  # Waxing 🌒
print(moon_phase("2025-07-25"))  # First Quarter 🌓



DEBUG: Date: 2025-07-05, Lunar Age: 9.56
Waxing Gibbous Moon 🌔
DEBUG: Date: 2025-07-12, Lunar Age: 16.56
Waning Gibbous Moon 🌖
DEBUG: Date: 2025-07-19, Lunar Age: 23.56
Waning Crescent Moon 🌘
DEBUG: Date: 2025-07-21, Lunar Age: 25.56
Waning Crescent Moon 🌘
DEBUG: Date: 2025-07-25, Lunar Age: 0.20
New Moon 🌑
